In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from fairBalance import *
from util import *

In [2]:
def balancing(X_train, target, knn, sensitive_attribute, features, drop_features, continous_features):
    fcb = fairBalance(X_train, features, continous_features, drop_features, sensitive_attribute, target, knn = knn)
    fcb.fit()
    X_balanced, y_balanced = fcb.generater()
    
    return X_balanced, y_balanced

In [3]:
def prediction(X_train, y_train, X_test, features):
    clf = LogisticRegression()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test[features])

    return y_pred

In [4]:
def evaluation(X_test, y_pred, target, sensitive_attribute, priority_group):
    y_test = X_test[target]

    df_pred = pd.DataFrame()
    df_pred[sensitive_attribute]=X_test[sensitive_attribute].tolist()
    df_pred['truth']=y_test.tolist()
    df_pred['pred']=y_pred
    print("---- overall performance ----")
    performance(df_pred)
    print("---- performance of different groups ----")
    group_comp(df_pred, sensitive_attribute, priority_group)

## An example with COMPAS dataset

#### 1. reading the data file

In [6]:
df=pd.read_csv('datasets/COMPAS.csv')

#### 2. set all required parameters

In [7]:
knn=5
target='two_year_recid'
drop_features=['race','sex']
features=list(set(df.keys().tolist())-set(drop_features+[target]))
continous_features=['age','juv_fel_count','juv_misd_count','juv_other_count','priors_count']
sensitive_attribute = 'race'
privileged_group = 1

#### 3. separate train-test sets

In [8]:
X_train, X_test = train_test_split(df, test_size=0.2, stratify=df[target], random_state=0)

#### 4. balance the train set

In [9]:
X_balanced, y_balanced = balancing(X_train, target, knn, sensitive_attribute, features, drop_features, continous_features)

Cluster the original dataset into 8 clusters:
Removing 1233 samples from the original dataset...


#### 5.prediction & performance evaluation

In [10]:
print("========= Performance before balancing ======== ")
y_pred = prediction(X_train[features], X_train[target], X_test, features)
evaluation(X_test, y_pred, target, sensitive_attribute, priority_group = 1)

print("======== Performance after balancing ======== ")
y_pred_bal = prediction(X_balanced, y_balanced, X_test, features)
evaluation(X_test, y_pred_bal, target, sensitive_attribute, priority_group = 1)

========= Performance before balancing ======== 
---- overall performance ----
Test Accuracy  0.6790923824959482
Precision: 0.678359
Recall: 0.679092
F1: 0.675811
---- performance of different groups ----
+----------------+-------+-------+-------+-------+
| Group          |    F1 |   TPR |   FPR |    PR |
|----------------+-------+-------+-------+-------|
| Privileged     | 0.670 | 0.443 | 0.164 | 0.272 |
| Non-privileged | 0.676 | 0.629 | 0.277 | 0.449 |
+----------------+-------+-------+-------+-------+
Equal Opportunity 0.186
Equal Odds 0.149
Statistical Parity 0.177
======== Performance after balancing ======== 
---- overall performance ----
Test Accuracy  0.6418152350081038
Precision: 0.640005
Recall: 0.641815
F1: 0.637885
---- performance of different groups ----
+----------------+-------+-------+-------+-------+
| Group          |    F1 |   TPR |   FPR |    PR |
|----------------+-------+-------+-------+-------|
| Privileged     | 0.637 | 0.437 | 0.220 | 0.304 |
| Non-privileged